In [1]:
# parsivel raw data processing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

names = np.array(['Date', 'Time', 'Intensity of precipitation (mm/h)',
               'Precipitation since start (mm)', 'Weather code SYNOP WaWa', 'Weather code METAR/SPECI',
               'Weather code NWS', 'Radar reflectivity (dBz)', 'MOR Visibility (m)',
               'Signal amplitude of Laserband', 'Number of detected particles', 'Temperature in sensor (ºC)',
               'Heating current (A)', 'Sensor voltage (V)', 'Optics status',
               'Kinetic Energy', 'Snow intensity (mm/h)'])
names = np.append(names, ['%.0f' % x for x in np.arange(1, 1025)])
names = np.append(names, ['</SPECTRUM>'])

# add .csv to original Parsivel data file 
file_name = ('2022-10-10-02-24')
df = pd.read_csv(file_name, sep=';', names=names, parse_dates=[['Date', 'Time']], index_col=['Date_Time'],
                 dayfirst=True, infer_datetime_format=True,na_values=[''])

# remove the repeating column names
# use .copy() to avoid warning of SettingWithCopy
# df = df.drop(df[df.iloc[:, 0] == df.iloc[0, 0]].index).copy()
try:
    df = df.drop(index='Date Time')
except:
    pass

# remove the last column
df = df.drop(['</SPECTRUM>'], axis=1)

# convert data type
df.index = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M:%S') #.tz_localize('Asia/Taipei')
df.loc[:, names[2]] = df.loc[:, names[2]].astype(float) # Intensity of precipitation (mm/h)

# replace <SPECTRUM> with the value of first DSD cell
df.loc[:, '1'] = df.loc[df.loc[:, '1'] != '<SPECTRUM>ZERO</SPECTRUM>', '1'].str[10:]

# fill NaN with 0.0, convert '' to float
df.loc[:,:] = df.loc[:,:].fillna(0.0)
df.loc[:, '1'] = pd.to_numeric(df.loc[:, '1'])

# resample by 1min
df = df.resample('1T', label='right', closed='right').sum()
pd.set_option('display.max_columns',None)

# Export data to a csv file
df.to_csv('2022-10-10-02-24_utc.csv')

C:\Users\88690\AppData\Local\Temp\ipykernel_11676\4282087987.py:19: DtypeWarning: Columns (1041) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name, sep=';', names=names, parse_dates=[['Date', 'Time']], index_col=['Date_Time'],
C:\Users\88690\AppData\Local\Temp\ipykernel_11676\4282087987.py:42: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, '1'] = pd.to_numeric(df.loc[:, '1'])
C:\Users\88690\AppData\Local\Temp\ipykernel_11676\4282087987.py:45: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1T', lab